Importing all necessary libraries

In [40]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

'Certified Fresh' , Genre :'Adventure_movies' : Rating : 'G'

In [44]:
def fetchUrl(url):
    response = requests.get(url)
    soup=BeautifulSoup(response.text, 'html.parser')
    return soup
    fetch_movie = [movie.strip() for movie in fetch_movie]

def scrapingData(soup,url):
    fetch_movie=soup.find_all('span',class_='p--small',attrs={'data-qa':"discovery-media-list-item-title"})
    fetch_rating=soup.find_all('rt-text',attrs={'slot':"criticsScore"})
    fetch_genre_resp=requests.get(url)
    fetch_genre=fetch_genre_resp.url[fetch_genre_resp.url.find('genres'):fetch_genre_resp.url.find('~ratings')]
    fetch_genre=fetch_genre[7:]
    movie_dict={'Title':[],'Rating':[],'Genre':[]}
    for index,movies in enumerate(fetch_movie):
        movie_dict['Title'].append(movies.text.strip())
        movie_dict['Rating'].append(fetch_rating[index].text)
        movie_dict['Genre'].append(fetch_genre)
    return movie_dict   

url="https://www.rottentomatoes.com/browse/movies_in_theaters/critics:certified_fresh~genres:adventure~ratings:g"
soup=fetchUrl(url)
movie_data=scrapingData(soup,url)
df=pd.DataFrame.from_dict(movie_data)
df.to_csv('movie_data.csv',index=False)